### Entrenamiento Modelo

In [1]:
import os
import numpy as np
import joblib
from deepface import DeepFace
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from tqdm import tqdm

# --- Configuración ---
DATA_DIR = "data"
CATEGORIES = ["neutral", "sonriendo"] # 0 = neutral, 1 = sonriendo
MODEL_NAME = "VGG-Face"
MODEL_FILENAME = "smile_classifier.pkl"
CATEGORIES_FILENAME = "categories.pkl"

X = []
y = []

print(f"Iniciando extracción de características con {MODEL_NAME}...")

# Recorrer categorías y procesar imágenes
for i, category in enumerate(CATEGORIES):
    path = os.path.join(DATA_DIR, category)
    image_files = os.listdir(path)
    print(f"\nProcesando categoría: {category} ({len(image_files)} imágenes)")
    
    for img_name in tqdm(image_files):
        img_path = os.path.join(path, img_name)
        
        try:
            # 'represent' detecta, alinea y extrae el embedding
            embedding_obj = DeepFace.represent(img_path=img_path, 
                                               model_name=MODEL_NAME, 
                                               enforce_detection=False)
            
            # embedding_obj es una lista, tomamos el primero
            embedding_vector = embedding_obj[0]['embedding']
            
            X.append(embedding_vector)
            y.append(i) # 0 para neutral, 1 para sonriendo
            
        except Exception as e:
            print(f"Error procesando {img_path}: {e}")

if not X:
    print("Error: No se extrajo ninguna característica. ¿Están las imágenes en la carpeta 'data'?")
    exit()

# Convertir a arrays de numpy
X = np.array(X)
y = np.array(y)

print(f"\nTotal de características extraídas: {X.shape}")
print(f"Total de etiquetas: {y.shape}")

# --- Evaluación (Opcional pero recomendado) ---
print("\nEvaluando modelo con split 80/20...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

eval_model = SVC(kernel='linear', probability=True)
eval_model.fit(X_train, y_train)

y_pred = eval_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=CATEGORIES))


# --- Entrenamiento del Modelo Final ---
print("\nEntrenando modelo final con TODOS los datos...")
final_model = SVC(kernel='linear', probability=True)
final_model.fit(X, y)

# Guardar el modelo
joblib.dump(final_model, MODEL_FILENAME)
# Guardar las categorías (para saber qué significa 0 y 1)
joblib.dump(CATEGORIES, CATEGORIES_FILENAME)

print(f"\n¡Éxito! Modelo guardado en '{MODEL_FILENAME}'")
print(f"Mapeo de categorías guardado en '{CATEGORIES_FILENAME}'")


Iniciando extracción de características con VGG-Face...

Procesando categoría: neutral (603 imágenes)


  0%|          | 0/603 [00:00<?, ?it/s]

25-11-13 16:08:37 - 🔗 vgg_face_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5 to C:\Users\dasum\.deepface\weights\vgg_face_weights.h5...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: C:\Users\dasum\.deepface\weights\vgg_face_weights.h5
100%|██████████| 580M/580M [00:20<00:00, 27.8MB/s]
100%|██████████| 603/603 [04:01<00:00,  2.49it/s]



Procesando categoría: sonriendo (600 imágenes)


100%|██████████| 600/600 [03:24<00:00,  2.93it/s]



Total de características extraídas: (1203, 4096)
Total de etiquetas: (1203,)

Evaluando modelo con split 80/20...
              precision    recall  f1-score   support

     neutral       0.97      0.97      0.97       121
   sonriendo       0.97      0.97      0.97       120

    accuracy                           0.97       241
   macro avg       0.97      0.97      0.97       241
weighted avg       0.97      0.97      0.97       241


Entrenando modelo final con TODOS los datos...

¡Éxito! Modelo guardado en 'smile_classifier.pkl'
Mapeo de categorías guardado en 'categories.pkl'


### Demo en vivo

In [9]:
import cv2
import joblib
import numpy as np
from deepface import DeepFace

# --- Configuración ---
MODEL_FILENAME = "smile_classifier.pkl"
CATEGORIES_FILENAME = "categories.pkl"
EMOJI_FILE = "happy.jpeg"
MODEL_NAME = "VGG-Face" # Debe ser el mismo usado en el entrenamiento
DETECTOR_BACKEND = "opencv" # 'mtcnn' o 'opencv'
CONF_THRESHOLD = 0.70 # Umbral de confianza para mostrar la reacción

# --- Helper para superponer emoji ---
def overlay_transparent(background_img, overlay_img, x, y):
    """ Superpone una imagen (con canal alfa) sobre otra. """
    try:
        # Asegurar que el overlay no se salga de los límites
        bg_h, bg_w, _ = background_img.shape
        ol_h, ol_w, ol_c = overlay_img.shape

        if x < 0: x = 0
        if y < 0: y = 0
        if x + ol_w > bg_w: ol_w = bg_w - x
        if y + ol_h > bg_h: ol_h = bg_h - y

        # Recortar el overlay si es necesario
        overlay_img = overlay_img[0:ol_h, 0:ol_w]

        if ol_c == 4: # Si tiene canal alfa
            alpha_s = overlay_img[:, :, 3] / 255.0
            alpha_l = 1.0 - alpha_s

            roi = background_img[y:y+ol_h, x:x+ol_w]

            for c in range(0, 3):
                roi[:, :, c] = (alpha_s * overlay_img[:, :, c] +
                                alpha_l * roi[:, :, c])

            background_img[y:y+ol_h, x:x+ol_w] = roi
    except Exception as e:
        print(f"Error al superponer imagen: {e}")
    
    return background_img

# --- Carga de recursos ---
try:
    model = joblib.load(MODEL_FILENAME)
    CATEGORIES = joblib.load(CATEGORIES_FILENAME)
except FileNotFoundError:
    print(f"Error: No se encontraron los archivos '{MODEL_FILENAME}' o '{CATEGORIES_FILENAME}'.")
    print("Por favor, ejecuta el script '02_entrenar_modelo.py' primero.")
    exit()

try:
    emoji = cv2.imread(EMOJI_FILE, -1) # -1 para cargar canal alfa
    if emoji is None: raise FileNotFoundError
except FileNotFoundError:
    print(f"Advertencia: No se encontró '{EMOJI_FILE}'. La reacción será solo texto.")
    emoji = None

print("Modelo cargado. Iniciando demo en vivo...")
print("Pulsa 'q' para salir.")

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    try:
        # 'represent' encuentra la cara y extrae el embedding en un solo paso
        embedding_objs = DeepFace.represent(frame,
                                          model_name=MODEL_NAME,
                                          detector_backend=DETECTOR_BACKEND,
                                          enforce_detection=False)
        
        # 'represent' devuelve una lista, tomamos el primer (y usualmente único) resultado
        if len(embedding_objs) > 0:
            obj = embedding_objs[0]
            embedding_vector = obj['embedding']
            
            facial_area = obj['facial_area']
            x = facial_area['x']
            y = facial_area['y']
            w = facial_area['w']
            h = facial_area['h']

            # Predecir con el modelo
            prediction = model.predict([embedding_vector])
            proba = model.predict_proba([embedding_vector])

            label_index = prediction[0]
            label_name = CATEGORIES[label_index]
            confidence = proba[0][label_index]
            
            # --- LA REACCIÓN ---
            if label_name == "sonriendo" and confidence > CONF_THRESHOLD:
                text = f"SONRIENDO ({confidence*100:.0f}%)"
                color = (0, 255, 0)
                
                # Reacción 1: Emoji
                if emoji is not None:
                    emoji_size = w // 2 # Tamaño del emoji relativo a la cara
                    resized_emoji = cv2.resize(emoji, (emoji_size, emoji_size))
                    # Posición (esquina superior derecha de la cara)
                    emoji_x = x + w - (emoji_size // 2)
                    emoji_y = y - (emoji_size // 2)
                    frame = overlay_transparent(frame, resized_emoji, emoji_x, emoji_y)
            else:
                text = f"{label_name.upper()} ({confidence*100:.0f}%)"
                color = (0, 0, 255)
            
            # Dibujar siempre el bounding box y el texto
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    except Exception as e:
        # A veces el detector falla
        print(f"Error en el bucle principal: {e}")
        pass

    cv2.imshow("Prototipo 1 - Detector de Sonrisas", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Modelo cargado. Iniciando demo en vivo...
Pulsa 'q' para salir.
